In [3]:
import pandas as pd
import numpy as np
import torch
import scipy.misc
import torchvision
from torchvision import utils, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
import csv

In [4]:
# Reading data to create id to label mapping
data = pd.read_csv("/scratch/tmp/WhaleClassData/train.csv")
label_set = set(data['Id'])
num_of_classes = len(label_set)
print("Number of classes: ", len(label_set))

label_to_id = {}
id_to_label = {}

def create_label_id_maps():
    _id = 0 
    for label in label_set: 
        label_to_id[label] = _id 
        id_to_label[_id] = label
        _id += 1

create_label_id_maps()

Number of classes:  4251


In [5]:
# Dataset class definition 
class WhaleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        if "train" in csv_file:
            self.whales = pd.read_csv(csv_file)
            self.test = 0
        else:
            self.whales = list(os.listdir(self.root_dir))
            self.test = 1
        self.transform = transform
    
    def __len__(self):
        return len(self.whales)
    
    def __getitem__(self, idx):
        if self.test:
            img_name = os.path.join(self.root_dir, self.whales[idx])
        else:
            img_name = os.path.join(self.root_dir, self.whales.iloc[idx, 0])
            
        image = Image.open(img_name).convert('RGB')
        if self.test:
            sample = {'image': image, 'image_name': self.whales[idx]}
        else:
            label = self.whales.iloc[idx, 1]
            label = label_to_id[label]
            sample = {'image': image, 'image_name': self.whales.iloc[idx, 0], 'label': label}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample

In [6]:
# Defining transformations to aid in data augmentation
resize_shape = (200, 200)
# resize_shape = (700, 1050)

transforms_1 = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize_shape),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(20, resample=Image.BILINEAR),
    torchvision.transforms.RandomAffine(30),
    torchvision.transforms.ToTensor()
])

transform_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize_shape),
    torchvision.transforms.ToTensor()
])

In [7]:
# Creating Train and Valid Datset samplers
whale_dataset_1 = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/train.csv", 
                             root_dir="/scratch/tmp/WhaleClassData/train",
                             transform=transform_resize)
whale_dataset_2 = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/train.csv", 
                             root_dir="/scratch/tmp/WhaleClassData/train",
                             transform=transforms_1)
whale_dataset_test = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/test", 
                             root_dir="/scratch/tmp/WhaleClassData/test/",
                             transform=transform_resize)

concat_dataset = torch.utils.data.ConcatDataset([whale_dataset_1, whale_dataset_2])

validation_split = .2
shuffle_dataset = True
random_seed = 42

dataset_size = len(concat_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [8]:
# Checking what kind of system you are using
try:
  import google.colab
  from google.colab import drive
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False
try:
    hostname = !hostname
    if 'lab' in hostname[0] and '.eng.utah.edu' in hostname[0]:
        IN_CADE = True
    else:
        IN_CADE = False
except:
    IN_CADE = False

assert(not IN_CADE or not IN_COLAB)

#defining the folders where datasets will be, depending on the system
machine_being_used = 'cade' if IN_CADE else ('colab' if IN_COLAB else 'other')
pre_folder = '/scratch/tmp/' if machine_being_used == 'cade' else './'

In [9]:
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x

In [10]:
define_gpu_to_use()

Chosen GPU: 0


In [19]:
# Model 1 - Resnet18 - Dataset
batch_size = 128

train_loader = torch.utils.data.DataLoader(concat_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(concat_dataset, batch_size=1, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(whale_dataset_test, batch_size=1, shuffle=False)

In [13]:
# Model 1 - Resnet18
num_epochs = 5

model_1 = models.resnet18(pretrained=True)
model_1.fc = torch.nn.Linear(in_features=512, out_features=num_of_classes, bias=True)
model_1.cuda()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=0.0001)
   
for epoch in range(num_epochs): 
    model_1.train() 
    losses = []
    print('Epoch ' + str(epoch)) 
    for sample in train_loader:
        optimizer.zero_grad() 
        images = sample['image'].cuda()
        label = sample['label'].cpu().numpy()
        label = label.reshape(label.shape[0],1)
        # Creating one hot vector
        y_onehot = torch.FloatTensor(label.shape[0], num_of_classes)
        y_onehot.zero_()
        y_onehot.scatter_(1, torch.LongTensor(label), 1)
        y_onehot = y_onehot.cuda()
        out = model_1(images)
        loss = criterion(out, y_onehot)
        loss.backward()
        optimizer.step() 
        losses.append(loss.item())
    print('loss: ' + str(np.mean(losses)))

Epoch 0
loss: 0.15845650209174042
Epoch 1
loss: 0.013211357091823894
Epoch 2
loss: 0.006716316272955268
Epoch 3
loss: 0.004608125572333172
Epoch 4
loss: 0.0036359487060127


In [33]:
# writeFile = open('submission.csv', 'w')
# writer = csv.writer(writeFile)
# writer.writerow(["Image", "Id"])

model_1.eval()
counter = 0
acc = 0

with torch.no_grad():
    for sample in validation_loader:
        images = sample['image'].cuda()
#         print (sample['image_name'][0])
#         print (counter)
#         counter += 1
        out = model_1(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][-5:]
#         ind_last = np.argsort(out)[0][-5:]
#         print(out)
#         print(out[0][ind[0]], " :: ", out[0][ind[1]])
#         print(out[0][ind_last[0]], " :: ", out[0][ind_last[1]])
#         print (int(sample['label'][0]))
#         print (ind[0])
        if int(sample['label'][0]) in ind : 
            acc += 1
#         if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
#             acc += 1
#         writer.writerow([sample['image_name'][0], id_to_label[ind[0]] + " " + id_to_label[ind[1]] + " " 
#                          + id_to_label[ind[2]] + " " + id_to_label[ind[3]] + " " + id_to_label[ind[4]]])


/home/u0931101/Work/DL_ImgAnal/env_dir/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [34]:
print(acc, len(validation_loader))
print ("Accuracy: %f" % (acc/len(validation_loader)*100))

348 3940
Accuracy: 8.832487


In [39]:
# write to the kaggle submission file 

print("# test files :: ", len(test_loader))
writeFile = open('submission.csv', 'w')
writer = csv.writer(writeFile)
writer.writerow(["Image", "Id"])

lines = [["Image", "Id"]]

model_1.eval()
counter = 0
acc = 0

with torch.no_grad():
    for sample in test_loader:
        images = sample['image'].cuda()
#         print (sample['image_name'][0])
#         print (counter)
        counter += 1
        out = model_1(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][-5:]
#         print(ind)
#         print (int(sample['label'][0]))
#         print (ind[0])
#         if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
#             acc += 1
        row = [sample['image_name'][0], id_to_label[ind[-1]] + " " + id_to_label[ind[-2]] + " " 
                         + id_to_label[ind[-3]] + " " + id_to_label[ind[-4]] + " " + id_to_label[ind[-5]]]
        lines.append(row)
        writer.writerow([sample['image_name'][0], id_to_label[ind[0]] + " " + id_to_label[ind[1]] + " " 
                         + id_to_label[ind[2]] + " " + id_to_label[ind[3]] + " " + id_to_label[ind[4]]])
    
print("# counter:: ", counter)

with open('submission_resnet18.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(lines)

#check how many rows are present in the file.
with open("submission.csv") as f:
    print("#rows in submission file :: ",sum(1 for line in f))
    
with open("submission_resnet18.csv") as f:
    print("#rows in submission_resnet18 file :: ",sum(1 for line in f))

# test files ::  15610


/home/u0931101/Work/DL_ImgAnal/env_dir/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# counter::  15610
#rows in submission file ::  15611
#rows in submission_resnet18 file ::  15611


In [44]:
vgg_mod = models.vgg11(pretrained=True)

vgg_mod.fc = torch.nn.Linear(in_features=1000, out_features=num_of_classes, bias=True)

vgg_mod

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [45]:
# Model 2 - Resnet18
num_epochs = 5

model_2 = models.vgg11(pretrained=True)
model_2.fc = torch.nn.Linear(in_features=1000, out_features=num_of_classes, bias=True)
model_2.cuda()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.0001)
   
for epoch in range(num_epochs): 
    model_2.train() 
    losses = []
    print('Epoch ' + str(epoch)) 
    for sample in train_loader:
        optimizer.zero_grad() 
        images = sample['image'].cuda()
        label = sample['label'].cpu().numpy()
        label = label.reshape(label.shape[0],1)
        # Creating one hot vector
        y_onehot = torch.FloatTensor(label.shape[0], num_of_classes)
        y_onehot.zero_()
        y_onehot.scatter_(1, torch.LongTensor(label), 1)
        y_onehot = y_onehot.cuda()
        out = model_2(images)
        loss = criterion(out, y_onehot)
        loss.backward()
        optimizer.step() 
        losses.append(loss.item())
    print('loss: ' + str(np.mean(losses)))

Epoch 0


RuntimeError: CUDA out of memory. Tried to allocate 625.00 MiB (GPU 0; 3.95 GiB total capacity; 2.81 GiB already allocated; 242.44 MiB free; 434.67 MiB cached)

In [ ]:
model.eval()
acc = 0

with torch.no_grad():
    for sample in validation_loader:
        images = sample['image'].cuda()
        out = model(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][:5]
        if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
            acc += 1

In [ ]:
print ("Accuracy: %f" % (acc/len(validation_loader)*100))

In [ ]:
# Model 3 - Resnet18
num_epochs = 5

model_2 = models.resnet18(pretrained=True)
model_2.fc = torch.nn.Linear(in_features=512, out_features=num_of_classes, bias=True)
model_2.cuda()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.001)
   
for epoch in range(num_epochs): 
    model_2.train() 
    losses = []
    print('Epoch ' + str(epoch)) 
    for sample in train_loader:
        optimizer.zero_grad() 
        images = sample['image'].cuda()
        label = sample['label'].cpu().numpy()
        label = label.reshape(label.shape[0],1)
        # Creating one hot vector
        y_onehot = torch.FloatTensor(label.shape[0], num_of_classes)
        y_onehot.zero_()
        y_onehot.scatter_(1, torch.LongTensor(label), 1)
        y_onehot = y_onehot.cuda()
        out = model_2(images)
        loss = criterion(out, y_onehot)
        loss.backward()
        optimizer.step() 
        losses.append(loss.item())
    print('loss: ' + str(np.mean(losses)))

In [ ]:
model_2.eval()
acc = 0

with torch.no_grad():
    for sample in validation_loader:
        images = sample['image'].cuda()
        out = model_2(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][:5]
        if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
            acc += 1

In [16]:
!nvidia-smi 

Sun May  5 14:57:46 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 970     Off  | 00000000:01:00.0  On |                  N/A |
| 11%   49C    P8    17W / 163W |   3804MiB /  4039MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  